In [1]:
import wandb
import pandas as pd
import numpy as np
import bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, Whisker
from bokeh.transform import factor_cmap, jitter
from bokeh.layouts import gridplot

In [2]:
api = wandb.Api()
runs = api.runs("hmr24/Grading-DiabeticRetinopathy-Comparisons-V3")

important_keys = ['lr', 'ema', 'swa', 'mixup', 'preprocessing', 'as_regression']
list_configs = []

for i, r in enumerate(runs):
    kappa = r.summary_metrics.get("Validation Quadratic Kappa", None)
    kappa_ddr = r.summary_metrics.get("Quadratic Kappa_DDR_test", None)
    kappa_eyepacs = r.summary_metrics.get("Quadratic Kappa_EYEPACS_test", None)
    kappa_idrid = r.summary_metrics.get("Quadratic Kappa_IDRID_test", None)
    if kappa is None:
        continue
    config = {k: v for k, v in r.config.items() if k in important_keys}
    config['name'] = r.name
    config['kappa'] = kappa
    config['kappa_ddr'] = kappa_ddr
    config['kappa_eyepacs'] = kappa_eyepacs
    config['kappa_idrid'] = kappa_idrid
    if i < 12:
        pass 
    else:
        list_configs.append(config)
    
df = pd.DataFrame(list_configs)

#df['ema'] = df['ema'].map({1: 'EMA', 0: 'No EMA'})
#df['swa'] = df['swa'].map({1: 'SWA', 0: 'No SWA'})
#df['mixup'] = df['mixup'].map({1: 'Mixup', 0: 'No Mixup'})
#df['as_regression'] = df['as_regression'].map({1: 'Regression', 0: 'Classification'})
df

,name,kappa,kappa_ddr,kappa_eyepacs,kappa_idrid
0,expert-glitter-166,0.957595,0.774146,0.726701,0.679320
1,eager-field-167,0.952968,0.770446,0.720667,0.718123
2,ruby-bee-168,0.960145,0.793029,0.716451,0.753644
3,stellar-pine-169,0.953048,0.762500,0.718422,0.712796
4,worthy-durian-170,0.949285,0.733957,0.683287,0.659757
5,legendary-blaze-171,0.949643,0.718678,0.669793,0.712313
6,usual-mountain-172,0.939620,0.700987,0.614180,0.660877
7,faithful-pyramid-173,0.958999,0.774140,0.719090,0.634716
8,polished-glade-174,0.948413,0.710607,0.678120,0.640334


In [3]:
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.palettes import MediumContrast3
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap
output_notebook()
preprocessing = ['absent', 'clahe_rgb', 'clahe_lab','clahe_mgg', 'autobalance', 'seoud',  'sarki', 'graham1','graham2']
databases_name = ['EYEPACS', 'IDRID', 'DDR']

data = {'preprocessing' : preprocessing,
        'EYEPACS'   : [],
        'IDRID'   : [],
        'DDR'   : []}

#ajout des valeurs kappa_test pour eyepacs issus de la dataframe
for kappa_test_val in df['kappa_eyepacs']:
    data['EYEPACS'].append(kappa_test_val)
#ajout des valeurs kappa_test pour idrid issus de la dataframe
for kappa_test_val in df['kappa_idrid']:
    data['IDRID'].append(kappa_test_val)
#ajout des valeurs kappa_test pour ddr issus de la dataframe
for kappa_test_val in df['kappa_ddr']:
    data['DDR'].append(kappa_test_val)

x = [ (preprocess, bdd) for preprocess in preprocessing for bdd in databases_name ]
data['preprocessing'].insert(0, data['preprocessing'].pop(2))
data['EYEPACS'].insert(0, data['EYEPACS'].pop(2))
data['IDRID'].insert(0, data['IDRID'].pop(2))
data['DDR'].insert(0, data['DDR'].pop(2))
data['preprocessing'].insert(3, data['preprocessing'].pop(7))
data['EYEPACS'].insert(3, data['EYEPACS'].pop(7))
data['IDRID'].insert(3, data['IDRID'].pop(7))
data['DDR'].insert(3, data['DDR'].pop(7))
counts = sum(zip(data['EYEPACS'], data['IDRID'], data['DDR']), ())

#change l'ordre de data en mettant clahe_max_green en position 3 ainsi que ses valeurs pour eyepacs ddr et idrid

source = ColumnDataSource(data=dict(x=x, counts=counts))

p = figure(x_range=FactorRange(*x), height=350, title="Quadratic Kappa test on different preprocessing",
           toolbar_location=None, tools="",output_backend="svg")

p.vbar(x='x', top='counts', width=0.9, source=source, line_color="white",
       fill_color=factor_cmap('x', palette=MediumContrast3, factors=databases_name, start=1, end=2))

p.y_range.start = 0.6
p.x_range.range_padding = 0.05
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)

Loading BokehJS ...